In [ ]:
#참고 : https://bigdata-doctrine.tistory.com/34

In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from fake_useragent import UserAgent
import datetime
from urllib import parse
from tqdm.notebook import tqdm
from time import sleep
import pandas as pd

URL = "https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1={}&sid2={}&date={}&page={}"
SID = [
    {'sid1': '100', 'sid2': '264', 'name1': '정치', 'name2': '대통령실'}, 
    {'sid1': '100', 'sid2': '265', 'name1': '정치', 'name2': '국회/정당'}, 
    {'sid1': '100', 'sid2': '268', 'name1': '정치', 'name2': '북한'}, 
    {'sid1': '100', 'sid2': '266', 'name1': '정치', 'name2': '행정'}, 
    {'sid1': '100', 'sid2': '267', 'name1': '정치', 'name2': '국방/외교'}, 
    {'sid1': '100', 'sid2': '269', 'name1': '정치', 'name2': '정치일반'}, 
    {'sid1': '101', 'sid2': '259', 'name1': '경제', 'name2': '금융'}, 
    {'sid1': '101', 'sid2': '258', 'name1': '경제', 'name2': '증권'}, 
    {'sid1': '101', 'sid2': '261', 'name1': '경제', 'name2': '산업/재계'}, 
    {'sid1': '101', 'sid2': '771', 'name1': '경제', 'name2': '중기/벤처'}, 
    {'sid1': '101', 'sid2': '260', 'name1': '경제', 'name2': '부동산'}, 
    {'sid1': '101', 'sid2': '262', 'name1': '경제', 'name2': '글로벌 경제'}, 
    {'sid1': '101', 'sid2': '310', 'name1': '경제', 'name2': '생활경제'}, 
    {'sid1': '101', 'sid2': '263', 'name1': '경제', 'name2': '경제 일반'}, 
    {'sid1': '102', 'sid2': '249', 'name1': '사회', 'name2': '사건사고'}, 
    {'sid1': '102', 'sid2': '250', 'name1': '사회', 'name2': '교육'}, 
    {'sid1': '102', 'sid2': '251', 'name1': '사회', 'name2': '노동'}, 
    {'sid1': '102', 'sid2': '254', 'name1': '사회', 'name2': '언론'}, 
    {'sid1': '102', 'sid2': '252', 'name1': '사회', 'name2': '환경'}, 
    {'sid1': '102', 'sid2': '59b', 'name1': '사회', 'name2': '인권/복지'}, 
    {'sid1': '102', 'sid2': '255', 'name1': '사회', 'name2': '식품/의료'}, 
    {'sid1': '102', 'sid2': '256', 'name1': '사회', 'name2': '지역'}, 
    {'sid1': '102', 'sid2': '276', 'name1': '사회', 'name2': '인물'}, 
    {'sid1': '102', 'sid2': '257', 'name1': '사회', 'name2': '사회 일반'}, 
    {'sid1': '103', 'sid2': '241', 'name1': '생활문화', 'name2': '건강정보'}, 
    {'sid1': '103', 'sid2': '239', 'name1': '생활문화', 'name2': '자동차/시승기'}, 
    {'sid1': '103', 'sid2': '240', 'name1': '생활문화', 'name2': '도로/교통'}, 
    {'sid1': '103', 'sid2': '237', 'name1': '생활문화', 'name2': '여행/레저'}, 
    {'sid1': '103', 'sid2': '238', 'name1': '생활문화', 'name2': '음식/맛집'}, 
    {'sid1': '103', 'sid2': '376', 'name1': '생활문화', 'name2': '패션/뷰티'}, 
    {'sid1': '103', 'sid2': '242', 'name1': '생활문화', 'name2': '공연/전시'}, 
    {'sid1': '103', 'sid2': '243', 'name1': '생활문화', 'name2': '책'}, 
    {'sid1': '103', 'sid2': '244', 'name1': '생활문화', 'name2': '종교'}, 
    {'sid1': '103', 'sid2': '248', 'name1': '생활문화', 'name2': '날씨'}, 
    {'sid1': '103', 'sid2': '245', 'name1': '생활문화', 'name2': '생활문화 일반'}, 
    {'sid1': '104', 'sid2': '231', 'name1': '세계', 'name2': '아시아/호주'}, 
    {'sid1': '104', 'sid2': '232', 'name1': '세계', 'name2': '미국/중남미'}, 
    {'sid1': '104', 'sid2': '233', 'name1': '세계', 'name2': '유럽'}, 
    {'sid1': '104', 'sid2': '234', 'name1': '세계', 'name2': '중동/아프리카'}, 
    {'sid1': '104', 'sid2': '322', 'name1': '세계', 'name2': '세계 일반'}, 
    {'sid1': '105', 'sid2': '731', 'name1': 'IT/과학', 'name2': '모바일'}, 
    {'sid1': '105', 'sid2': '226', 'name1': 'IT/과학', 'name2': '인터넷/SNS'}, 
    {'sid1': '105', 'sid2': '227', 'name1': 'IT/과학', 'name2': '통신/뉴미디어'}, 
    {'sid1': '105', 'sid2': '230', 'name1': 'IT/과학', 'name2': 'IT 일반'}, 
    {'sid1': '105', 'sid2': '732', 'name1': 'IT/과학', 'name2': '보안/해킹'}, 
    {'sid1': '105', 'sid2': '283', 'name1': 'IT/과학', 'name2': '컴퓨터'}, 
    {'sid1': '105', 'sid2': '229', 'name1': 'IT/과학', 'name2': '게임/리뷰'}, 
    {'sid1': '105', 'sid2': '228', 'name1': 'IT/과학', 'name2': '과학 일반'},
]
newlines = {
    "br", "div", "p"
}
DAY = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=1)
DAY = DAY.strftime('%Y%m%d')
UA = UserAgent().random

def Crawling(SID, URL, DAY, UA):
    url_list = []
    for sid in SID:
        url = URL.format(sid['sid1'], sid['sid2'], DAY, '10')
        url_list.append(url)
    return url
        
def art_crawl(url):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        urls(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    # art_dic = {}
    
    ## 1.
    # title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    # url = urls[sid][index]
    # url = urls['url']
    html = requests.get(url, headers={"User-Agent":  "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0"}, verify=False)
    soup = BeautifulSoup(html.text, "lxml")
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    return date_str, main_str


""" 뉴스 페이지 목록 조회 """
def news_list_page(date):
    keys = set()
    urls = []
    sid1set = []
    sid2set = []
    dataset_set = {}
    # 뉴스테마별 딕셔너리
    urlset = []
    for sid in SID:
        #urlset[sid["sid1"], sid["sid2"]] = []
        # max page ?
        for page in range(1,999):
            sid1, sid2, page = sid["sid1"], sid["sid2"], page
            key = f"{sid1}.{sid2}.{page}"
            if key not in keys:
                keys.add(key)
                sid1set.append(sid1)
                sid2set.append(sid2)
                urlset.append(URL.format(sid1, sid2, date, page))
    dataset_set['sid1'] = sid1set
    dataset_set['sid2'] = sid2set
    dataset_set['link'] = urlset
    
    dataset_sid1 = []
    dataset_sid2 = []
    dataset_link = []
                
    index = 0
    
    DF_set = pd.DataFrame(dataset_set)

    for i,j in tqdm(DF_set.groupby(['sid1','sid2'])[['link']]):
        sid_main = []
        sid_sub = []
        url_main = []
        break_key = []
        print(i)
        # 각 세부테마별 중복 url 존재시 멈춤 (ex. 마지막 페이지 다음 페이지에서는 url이 겹치므로 정지)
        for sid1, sid2, url in (zip(j['sid1'],j['sid2'],j['link'])):
            html = requests.get(url, headers={"User-Agent":  "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0"}, verify=False)
            soup = BeautifulSoup(html.text, "lxml")
            #name1, name2 = names[index][0], names[index][1]       
            """ 뉴스 페이지 목록 조회 """
            items = soup.select("#main_content > div.list_body.newsflash_body > ul > li > dl > dt[class!='photo'] > a")
            for item in tqdm(items):
                url, title = item["href"], item.text.strip()  
                if url not in url_main:
                    if 0 < len(title):   
                        sid_main.append(sid1)
                        sid_sub.append(sid2)
                        url_main.append(url)
                else:
                    # 메인섹션 url 수집 중단 key
                    break_key.append(1)
                    break
            if len(break_key) == 0:
                items = url_main
            
                if len(items) != 0:
                    dataset_sid1.extend(sid_main)
                    dataset_sid2.extend(sid_sub)
                    dataset_link.extend(items)
                    #print(f"========={names[index][0], names[index][1]}=========")
                    #sleep(1)
                    index = index + 1
                else:
                    index = index + 1
            else:
                break
        print("=============================================수집완료======================================================")
        
    return dataset_sid1, dataset_sid2, dataset_link

In [4]:
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

dum1, dum2, dum3 = news_list_page('20230903')

  0%|          | 0/48 [00:00<?, ?it/s]

('100', '264')


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

=============================================수집완료======================================================
('105', '732')


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

=============================================수집완료======================================================


In [5]:
df = pd.DataFrame([x for x in zip(dum1, dum2, dum3)], columns = ['sid1','sid2','url'])
df = df.drop_duplicates()

In [6]:
for i,j in df[:4].iterrows():
    print(j['url'])

https://n.news.naver.com/mnews/article/003/0012066622?sid=100
https://n.news.naver.com/mnews/article/421/0007025987?sid=100
https://n.news.naver.com/mnews/article/020/0003518442?sid=100
https://n.news.naver.com/mnews/article/003/0012066564?sid=100


In [ ]:
def art_crawl(url):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

dataset = []    
for i,j in tqdm(df.iterrows()):
    art_dic = art_crawl(j['url'])
    if 0 < len(art_dic["title"]):
        dataset.append({"sid1": j['sid1'], "sid2": j['sid2'], "url": j['url'], "title": art_dic["title"], "date": art_dic["date"], "main": art_dic["main"]})

0it [00:00, ?it/s]

In [ ]:
import pandas as pd

art_df = pd.DataFrame(dataset)

In [165]:
art_df

,sid1,sid2,url,title,date,main
0,100,264,https://n.news.naver.com/mnews/article/003/001...,국정원 '방산침해대응협의회' 11일 출범…방산업계 공조 강화,2023.09.03. 오후 11:57,안보실·국방부·방사청…방산기업 15개 참여\n\n\n\n국정원 로고 (사진=국정원 ...
1,100,264,https://n.news.naver.com/mnews/article/421/000...,"당정 ""교권회복 4법·추가입법에 속도""…'공교육 멈춤' 긴급논의(종합)",2023.09.03. 오후 10:27,"총리공관서 고위당정…윤재옥·이주호 '49재 추모제' 참석""교권회복 4법 외 교사들과..."
2,100,264,https://n.news.naver.com/mnews/article/020/000...,"젊어진 행안부, 실·국장 64% 교체…이상민 “재출범 수준으로 거듭나야”",2023.09.03. 오후 8:20,이상민 행정안전부 장관. 뉴시스행정안전부(행안부)가 본부 실·국장의 절반이 넘는 6...
3,100,264,https://n.news.naver.com/mnews/article/003/001...,"젊어진 행안부, 실·국장 64% 교체…국정과제 추진 고삐",2023.09.03. 오후 7:47,"'조직 쇄신' 기수 파괴·보직경로 타파이상민 장관 ""재출범 수준 거듭나야""\n\n\..."
4,100,264,https://n.news.naver.com/mnews/article/014/000...,"인니·인도 가는 尹대통령, 세일즈외교 주력",2023.09.03. 오후 7:30,수출 확대·공급망 협력 시너지 기대尹정부서 순방 계기 5조6천억 유치 윤석열 대통...
...,...,...,...,...,...,...
8789,105,732,https://n.news.naver.com/mnews/article/014/000...,빅데이터 혁신 융합 대학사업단 X 신한금융그룹 빅데이터 해커톤 대회 ‘제2회 신-빅...,2023.09.01. 오후 1:56,빅데이터 혁신 융합 대학사업단 X 신한금융그룹 빅데이터 해커톤 대회 ‘제2회 신-빅...
8790,105,732,https://n.news.naver.com/mnews/article/138/000...,"소만사, 외산 가상화 솔루션 윈복 7회째… 누적 라이선스 2만 유저 이상",2023.09.01. 오후 1:39,ⓒ소만사[디지털데일리 이종현기자] 사이버보안 기업 소만사는 PC 가상화 솔루션 ‘브...
8791,105,732,https://n.news.naver.com/mnews/article/138/000...,"KISA, 규범준수 경영시스템 국제표준 인증 획득",2023.09.01. 오후 1:34,전라남도 나주시에 위치한 KISA 본원[디지털데일리 이종현기자] 한국인터넷진흥원(K...
8792,105,732,https://n.news.naver.com/mnews/article/138/000...,"KISIA, 국방보안콘퍼런스 부대행사로 보안 제품 전시회 개최",2023.09.01. 오전 11:41,ⓒKISA[디지털데일리 이종현기자] 한국정보보호산업협회(KISIA)는 9월6일 진행...


In [166]:
# 출력 옵션 설정
# pd.set_option("display.max_rows", None)  # 모든 행 표시
pd.set_option("display.max_columns", None)  # 모든 열 표시
pd.set_option("display.width", None)  # 줄 바꿈 없이 전체 내용 표시

In [167]:
# 텍스트 전처리
def text_preprocessing(text):
    text = text.replace('\n', '')
    text = text.replace('   ', ' ')
    text = text.replace('\\', '')
    return text

In [168]:
art_df['main'] = art_df['main'].apply(text_preprocessing)

In [169]:
from datetime import datetime

def to_dt_time(date):
    if date != '':
        date = datetime.strptime(date[:10], '%Y.%m.%d')
        date = date.strftime('%Y-%m-%d')
        return date
    else:
        return date

In [170]:
# 날짜
now = datetime.now()
now = now.date().strftime('%Y-%m-%d')

art_df['ymd'] = art_df['date'].apply(to_dt_time)

In [171]:
art_df

,sid1,sid2,url,title,date,main,ymd
0,100,264,https://n.news.naver.com/mnews/article/003/001...,국정원 '방산침해대응협의회' 11일 출범…방산업계 공조 강화,2023.09.03. 오후 11:57,안보실·국방부·방사청…방산기업 15개 참여국정원 로고 (사진=국정원 제공) *재판매...,2023-09-03
1,100,264,https://n.news.naver.com/mnews/article/421/000...,"당정 ""교권회복 4법·추가입법에 속도""…'공교육 멈춤' 긴급논의(종합)",2023.09.03. 오후 10:27,"총리공관서 고위당정…윤재옥·이주호 '49재 추모제' 참석""교권회복 4법 외 교사들과...",2023-09-03
2,100,264,https://n.news.naver.com/mnews/article/020/000...,"젊어진 행안부, 실·국장 64% 교체…이상민 “재출범 수준으로 거듭나야”",2023.09.03. 오후 8:20,이상민 행정안전부 장관. 뉴시스행정안전부(행안부)가 본부 실·국장의 절반이 넘는 6...,2023-09-03
3,100,264,https://n.news.naver.com/mnews/article/003/001...,"젊어진 행안부, 실·국장 64% 교체…국정과제 추진 고삐",2023.09.03. 오후 7:47,"'조직 쇄신' 기수 파괴·보직경로 타파이상민 장관 ""재출범 수준 거듭나야""[세종=뉴...",2023-09-03
4,100,264,https://n.news.naver.com/mnews/article/014/000...,"인니·인도 가는 尹대통령, 세일즈외교 주력",2023.09.03. 오후 7:30,수출 확대·공급망 협력 시너지 기대尹정부서 순방 계기 5조6천억 유치 윤석열 대통...,2023-09-03
...,...,...,...,...,...,...,...
8789,105,732,https://n.news.naver.com/mnews/article/014/000...,빅데이터 혁신 융합 대학사업단 X 신한금융그룹 빅데이터 해커톤 대회 ‘제2회 신-빅...,2023.09.01. 오후 1:56,빅데이터 혁신 융합 대학사업단 X 신한금융그룹 빅데이터 해커톤 대회 ‘제2회 신-빅...,2023-09-01
8790,105,732,https://n.news.naver.com/mnews/article/138/000...,"소만사, 외산 가상화 솔루션 윈복 7회째… 누적 라이선스 2만 유저 이상",2023.09.01. 오후 1:39,ⓒ소만사[디지털데일리 이종현기자] 사이버보안 기업 소만사는 PC 가상화 솔루션 ‘브...,2023-09-01
8791,105,732,https://n.news.naver.com/mnews/article/138/000...,"KISA, 규범준수 경영시스템 국제표준 인증 획득",2023.09.01. 오후 1:34,전라남도 나주시에 위치한 KISA 본원[디지털데일리 이종현기자] 한국인터넷진흥원(K...,2023-09-01
8792,105,732,https://n.news.naver.com/mnews/article/138/000...,"KISIA, 국방보안콘퍼런스 부대행사로 보안 제품 전시회 개최",2023.09.01. 오전 11:41,ⓒKISA[디지털데일리 이종현기자] 한국정보보호산업협회(KISIA)는 9월6일 진행...,2023-09-01


In [193]:
SID[0]['sid1']

'100'

In [208]:
SID_df = pd.DataFrame(SID)

In [224]:
df_tot = pd.merge(art_df, SID_df, on = ['sid1', 'sid2'], how = 'inner')

In [225]:
df_tot

,sid1,sid2,url,title,date,main,ymd,name1,name2
0,100,264,https://n.news.naver.com/mnews/article/003/001...,국정원 '방산침해대응협의회' 11일 출범…방산업계 공조 강화,2023.09.03. 오후 11:57,안보실·국방부·방사청…방산기업 15개 참여국정원 로고 (사진=국정원 제공) *재판매...,2023-09-03,정치,대통령실
1,100,264,https://n.news.naver.com/mnews/article/421/000...,"당정 ""교권회복 4법·추가입법에 속도""…'공교육 멈춤' 긴급논의(종합)",2023.09.03. 오후 10:27,"총리공관서 고위당정…윤재옥·이주호 '49재 추모제' 참석""교권회복 4법 외 교사들과...",2023-09-03,정치,대통령실
2,100,264,https://n.news.naver.com/mnews/article/020/000...,"젊어진 행안부, 실·국장 64% 교체…이상민 “재출범 수준으로 거듭나야”",2023.09.03. 오후 8:20,이상민 행정안전부 장관. 뉴시스행정안전부(행안부)가 본부 실·국장의 절반이 넘는 6...,2023-09-03,정치,대통령실
3,100,264,https://n.news.naver.com/mnews/article/003/001...,"젊어진 행안부, 실·국장 64% 교체…국정과제 추진 고삐",2023.09.03. 오후 7:47,"'조직 쇄신' 기수 파괴·보직경로 타파이상민 장관 ""재출범 수준 거듭나야""[세종=뉴...",2023-09-03,정치,대통령실
4,100,264,https://n.news.naver.com/mnews/article/014/000...,"인니·인도 가는 尹대통령, 세일즈외교 주력",2023.09.03. 오후 7:30,수출 확대·공급망 협력 시너지 기대尹정부서 순방 계기 5조6천억 유치 윤석열 대통...,2023-09-03,정치,대통령실
...,...,...,...,...,...,...,...,...,...
8789,105,732,https://n.news.naver.com/mnews/article/014/000...,빅데이터 혁신 융합 대학사업단 X 신한금융그룹 빅데이터 해커톤 대회 ‘제2회 신-빅...,2023.09.01. 오후 1:56,빅데이터 혁신 융합 대학사업단 X 신한금융그룹 빅데이터 해커톤 대회 ‘제2회 신-빅...,2023-09-01,IT/과학,보안/해킹
8790,105,732,https://n.news.naver.com/mnews/article/138/000...,"소만사, 외산 가상화 솔루션 윈복 7회째… 누적 라이선스 2만 유저 이상",2023.09.01. 오후 1:39,ⓒ소만사[디지털데일리 이종현기자] 사이버보안 기업 소만사는 PC 가상화 솔루션 ‘브...,2023-09-01,IT/과학,보안/해킹
8791,105,732,https://n.news.naver.com/mnews/article/138/000...,"KISA, 규범준수 경영시스템 국제표준 인증 획득",2023.09.01. 오후 1:34,전라남도 나주시에 위치한 KISA 본원[디지털데일리 이종현기자] 한국인터넷진흥원(K...,2023-09-01,IT/과학,보안/해킹
8792,105,732,https://n.news.naver.com/mnews/article/138/000...,"KISIA, 국방보안콘퍼런스 부대행사로 보안 제품 전시회 개최",2023.09.01. 오전 11:41,ⓒKISA[디지털데일리 이종현기자] 한국정보보호산업협회(KISIA)는 9월6일 진행...,2023-09-01,IT/과학,보안/해킹


In [226]:
df_tot[['name1', 'name2','url']].groupby(['name1', 'name2']).count()

url
name1 name2        
IT/과학 IT 일반     195
      게임/리뷰      20
      과학 일반      45
      모바일        27
      보안/해킹      19
      인터넷/SNS    48
      컴퓨터        20
      통신/뉴미디어    56
경제    경제 일반     940
      글로벌 경제     41
      금융        224
      부동산       160
      산업/재계     611
      생활경제      194
      중기/벤처      52
      증권        138
사회    교육        402
      노동         88
      사건사고      325
      사회 일반     416
      식품/의료      71
      언론         18
      인권/복지      39
      인물        118
      지역       1701
      환경         56
생활문화  건강정보       52
      공연/전시      90
      날씨        108
      도로/교통      20
      생활문화 일반   318
      여행/레저      71
      음식/맛집      20
      자동차/시승기    89
      종교         20
      책          21
      패션/뷰티      20
세계    미국/중남미     82
      세계 일반     197
      아시아/호주    126
      유럽         88
      중동/아프리카    20
정치    국방/외교     147
      국회/정당     371
      대통령실       31
      북한        109
      정치일반      696
      행정         74

In [227]:
df_tot.to_csv("df_tot.csv")